In [1]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import hashlib
import random

In [3]:
def mdFiveIt(string):
    
    str = string
    result = hashlib.md5(str.encode()) 
    #print("The hexadecimal equivalent of hash is : ", end ="") 
    #print(result.hexdigest()) 
    
    return result.hexdigest()

In [4]:
def printelement(elem):
    element_text = elem.text
    element_attribute_value = elem.get_attribute('value')

    #print(elem)
    print('element.text: {0}'.format(element_text))
    #print('element.get_attribute(\'value\'): {0}'.format(element_attribute_value))


In [5]:
class ElementHolder(object):
    def __init__(self, name, element):
        self.name = name
        self.element = element
def simpleScan(title,url,elementsToScan = ["button","a"]):
    driver = webdriver.Firefox()
    driver.get(url)
    #assert "BIG small Games" in driver.title
    html = driver.page_source
    print("MD5 encrypted source page: ",mdFiveIt(html))
    print()
    print("MD5 encrypted url: ",mdFiveIt(driver.current_url))
    print()
    print("URL: ",driver.current_url)
    print()
    elements = []
    
    for name in elementsToScan:
        element = driver.find_elements_by_tag_name(name)
        elements.append(ElementHolder(name,element))
        
    for i in elements:
        print(i.name,": ")
        for j in i.element:
            printelement(j)
        print()    
    
    assert "No results found." not in driver.page_source
    driver.close()
simpleScan("BIG small Games","file:///C:/Users/IAmir%20Ingher/Documents/projekti/school/ai/reinf_ai/project/html_generator/test/test1.html",)    

MD5 encrypted source page:  d894a8fbac67a608740cbe88ef14922a

MD5 encrypted url:  74fe24c4ee286cc6b74e7f9fbf775e1c

URL:  file:///C:/Users/IAmir%20Ingher/Documents/projekti/school/ai/reinf_ai/project/html_generator/test/test1.html

button : 

a : 
element.text: Visit our HTML 1_1
element.text: Visit our HTML 2_2
element.text: Visit our HTML 3_3
element.text: Visit our HTML 4_4
element.text: Visit our HTML 5_5



In [6]:
class hashState(object):
    def __init__(self, hash_value, state_num):
        self.hash_value = hash_value
        self.state_num = state_num
        

In [7]:
class Agent(object):
    steps = 0
    def __init__(self, startPos,):
        self.startPos = startPos
        self.pos = startPos
        self.history = []

In [8]:
class State(object):
    def __init__(self, num, element):
        self.num = num
        self.element = element
        self.stateValue = -1

In [9]:
class Model(object):
    def __init__(self, mdhash, url, states):
        self.mdhash = mdhash
        self.url = url
        self.states = states

In [10]:
class Models(object):
    def __init__(self, models):
        self.mdhash = models
        

In [11]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(driver,xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [12]:
def scrollAllElements(elements):
    for element in elements:
        printelement(element)

In [13]:
def randomListClick(driver,elements,dontVisit,url):
    
    
    if(len(elements) > 0):
        random_num = random.randint(0,len(elements)-1)
        element=elements[random_num]
        for t in dontVisit:
            if(element.text == t):
                return random_num
        #print("clicked: ",element.text)    
        element.click()
        return random_num
    driver.get(url)
    return -1

In [14]:
def isThisNewModel(models, md5):
    for model in models:
        if(model.mdhash == md5):
            return False
    return True                   

In [15]:
def generateStates(elements):
    states = []
    count = 0
    for element in elements:
        states.append(State(count, element))
        count += 1
    return states

In [16]:
def printModel(m):
    print()
    print("Explored Model(s):")
    for model in m:
        print()
        print(model.url[-14:]," ",model.mdhash,": ", end="")
        #print(model.mdhash,": ", end="")
        for s in model.states:
            with np.printoptions(precision=2):
                print(s.num,"(","%.2f" % round(s.stateValue,5),") ", end="")
                #print(s.num,"(",s.stateValue,") ", end="")

In [17]:
def MC_button_demo(max_iteration,url,goalXpath,dontVisit,element_tag="button"):
    model = []
    lenght = []
    driver = webdriver.Firefox()
    driver.get(url)
    
    
    for i in range(1,max_iteration):
        driver.get(url)
        agent = Agent(mdFiveIt(driver.current_url))
        while(True):
            
            if(check_exists_by_xpath(driver,goalXpath)):
                break
            else:
                elements = driver.find_elements_by_tag_name(element_tag)
                
                
                md5 = mdFiveIt(driver.current_url)
                if(isThisNewModel(model,md5)):
                    model.append(Model(md5,driver.current_url,generateStates(elements)))
                    
                selectedState = randomListClick(driver,elements,dontVisit,url)
                if(selectedState != -1):
                    agent.history.append(hashState(md5,selectedState))
                    
                #print(driver.current_url[-14:])
        steps =1
        agent.history.reverse()
        for visitedValues in agent.history:
            
            for m in model:
                
                if(m.mdhash == visitedValues.hash_value):
                    mean =  m.states[visitedValues.state_num].stateValue - steps
                    m.states[visitedValues.state_num].stateValue = mean
                    
                    steps +=1
            
    driver.close()
    return model

In [18]:
def calculateStatesNewValue(model,visitedValue,oldVisitedValue,a=0.8,r=-1):
    #value = old_state + a*(r + new_state - old_state)
    mean = 0
    if (oldVisitedValue.state_num == None or visitedValue.state_num == None):
        return
    old = 0
    for m in model:
        if(m.mdhash == visitedValue.hash_value):
            new = m.states[visitedValue.state_num].stateValue
            updateModel = m
            
        if(m.mdhash == oldVisitedValue.hash_value):
            old = m.states[oldVisitedValue.state_num].stateValue
    
    mean = old + a*(r + new - old)
    
    updateModel.states[visitedValue.state_num].stateValue = mean                                                                   

In [19]:
def TD_button_demo(max_iteration,url,goalXpath,dontVisit,element_tag="button"):
    model = []
    lenght = []
    driver = webdriver.Firefox()
    driver.get(url)
    
    visitedValue= hashState("this is not page",0)
    for i in range(1,max_iteration):
        driver.get(url)
        agent = Agent(mdFiveIt(driver.current_url))
        while(True):
            
            if(check_exists_by_xpath(driver,goalXpath)):
                break
            else:
                elements = driver.find_elements_by_tag_name(element_tag)
                
                
                md5 = mdFiveIt(driver.current_url)
                if(isThisNewModel(model,md5)):
                    model.append(Model(md5,driver.current_url,generateStates(elements)))
                    
                  
                selectedState = randomListClick(driver,elements,dontVisit,url)
                if(selectedState != -1):
                    oldVisitedValue = visitedValue
                    visitedValue = hashState(md5,selectedState)
                    
                    calculateStatesNewValue(model, visitedValue, oldVisitedValue,)
                else:
                    oldVisitedValue = visitedValue
                    visitedValue = hashState(md5,selectedState)
                    print("deb")
                    calculateStatesNewValue(model, visitedValue, oldVisitedValue,a=0.8,r=1)
                    
                    
                
                
                #print(driver.current_url[-14:])
        steps =1
        agent.history.reverse()
        
            
    driver.close()
    return model

In [79]:
dontVisit =['download link','TWITTER','AVARA']
model = TD_button_demo(20,"file:///C:/Users/IAmir%20Ingher/Documents/projekti/school/ai/reinf_ai/project/html_generator/test/test1.html","//*[contains(text(),'goaaaaaaaalllll')]",dontVisit,"a")
printModel(model)

deb


IndexError: list index out of range

In [27]:
def navigateToGoal(model):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-27-c0555fe2dc0b>, line 2)

In [33]:
dontVisit =['download link','TWITTER','AVARA']
model = MC_button_demo(10,"http://myy.haaga-helia.fi/~a1602639/ehot/","//*[contains(text(),'Pink Team')]",dontVisit,)
printModel(model)

clicked:  Pizzeon
http://myy.haaga-helia.fi/~a1602639/ehot/pizzeon.html
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Login
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Hobsheim
http://myy.haaga-helia.fi/~a1602639/ehot/hobsheim.html
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Pizzeon
http://myy.haaga-helia.fi/~a1602639/ehot/pizzeon.html
http://myy.haaga-helia.fi/~a1602639/ehot/
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Login
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Login
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Hobsheim
http://myy.haaga-helia.fi/~a1602639/ehot/hobsheim.html
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Hobsheim
http://myy.haaga-helia.fi/~a1602639/ehot/hobsheim.html
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Hobsquabble
http://myy.haaga-helia.fi/~a1602639/ehot/hobsquabble.html
http://myy.haaga-helia.fi/~a1602639/ehot/
clicked:  Login
http://myy.haaga-helia.fi/~a1602639/ehot/
http://myy.haaga-helia.fi/~a

In [49]:
model = MC_button_demo(100,"file:///C:/Users/IAmir%20Ingher/Documents/projekti/school/ai/reinf_ai/project/html_generator/test/test1.html","//*[contains(text(),'goaaaaaaaalllll')]",dontVisit,"a")
printModel(model)


Explored Model(s):

est/test1.html   74fe24c4ee286cc6b74e7f9fbf775e1c : 0 ( -8965.00 ) 1 ( -9432.00 ) 2 ( -8973.00 ) 3 ( -6905.00 ) 4 ( -10825.00 ) 
t/test1_4.html   5108f7c9b78cb49b18c1e2a8fceed21a : 0 ( -3133.00 ) 1 ( -3260.00 ) 2 ( -199.00 ) 
4/test4_1.html   d124f40e3a0d2128be87f9687e8e5f06 : 
t/test1_5.html   4f0b076679a1a195f263a5196014d9b5 : 0 ( -5189.00 ) 1 ( -5294.00 ) 
5/test5_2.html   7a67112891f4be246be19c67bb5fed1b : 0 ( -5133.00 ) 
test5_2_1.html   4643cbd2ae15203ddf947073f4f53fdd : 
t/test1_2.html   bff445a5e66d8375b05580b9f4107b46 : 0 ( -5084.00 ) 1 ( -4034.00 ) 
2/test2_1.html   06e325830bc18c16138c9966d6dd8e5e : 
t/test1_3.html   cb2f1088ae7b95871cf144a9206fcae7 : 0 ( -8666.00 ) 
3/test3_1.html   24d6454160922381be56bb1b31fc620b : 
t/test1_1.html   f7f45e9224cec48d185d04bd2626c8bb : 
5/test5_1.html   8f15b8b4bf7f6e9b2f2ac73f85f3352a : 
2/test2_2.html   a673ea75c516acc85ecb9e217b35ed59 : 
4/test4_3.html   edbb2f8e8da05583810a9a3d8a2a7188 : 0 ( -100.00 ) 
4/test4_2.htm